In [1]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import average_precision_score, make_scorer, roc_auc_score
import optuna

# ============================================
# 1. Load data
# ============================================
file_path = "/kaggle/input/income-case-study/income_case_study.csv"  # <-- change if needed
df_raw = pd.read_csv(file_path, sep=";")

# ============================================
# 2. Basic cleaning
# ============================================
for col in df_raw.select_dtypes(include="object").columns:
    df_raw[col] = df_raw[col].str.strip()

df = df_raw.copy()

# ============================================
# 3. Missing value handling
# ============================================
# Age & education-num flags and median imputation
df["age_missing_flag"] = df["age"].isna().astype("int8")
df["edu_num_missing_flag"] = df["education-num"].isna().astype("int8")

df["age"] = df["age"].fillna(df["age"].median()).astype("int64")
df["education-num"] = df["education-num"].fillna(df["education-num"].median()).astype("int64")

# Education level text missing -> "Unknown"
df["education_level"] = df["education_level"].fillna("Unknown")

# ============================================
# 4. Feature engineering
# ============================================

# 4.1 Age & hours buckets
age_bins = [0, 24, 34, 44, 54, 64, np.inf]
age_labels = ["<25", "25-34", "35-44", "45-54", "55-64", "65+"]
df["age_group"] = pd.cut(df["age"], bins=age_bins, labels=age_labels, right=True)

hours_bins = [0, 34, 40, 45, 60, np.inf]
hours_labels = ["<35", "35-40", "41-45", "46-60", "60+"]
df["hours_group"] = pd.cut(df["hours-per-week"], bins=hours_bins, labels=hours_labels, right=True)

df["long_hours_flag"] = (df["hours-per-week"] >= 46).astype(int)

# 4.2 Education score & high_education_flag
edu_order = [
    "Preschool","1st-4th","5th-6th","7th-8th","9th","10th","11th","12th",
    "HS-grad","Some-college","Assoc-voc","Assoc-acdm",
    "Bachelors","Masters","Prof-school","Doctorate","Unknown"
]
edu_map = {e: i for i, e in enumerate(edu_order)}
df["education_score"] = df["education_level"].map(edu_map).fillna(-1).astype(int)

high_edu_levels = {"Bachelors","Masters","Prof-school","Doctorate"}
df["high_education_flag"] = df["education_level"].isin(high_edu_levels).astype(int)

# 4.3 Capital features
df["has_capital_gain"] = (df["capital-gain"] > 0).astype(int)
df["has_capital_loss"] = (df["capital-loss"] > 0).astype(int)
df["log_capital_gain"] = np.log1p(df["capital-gain"])
df["log_capital_loss"] = np.log1p(df["capital-loss"])

# Buckets: 0, 1–1000, 1001–10000, >10000
cap_gain_bins = [-0.5, 0.5, 1000, 10000, np.inf]
cap_gain_labels = ["0", "1-1000", "1001-10000", ">10000"]
df["capital_gain_bucket"] = pd.cut(
    df["capital-gain"],
    bins=cap_gain_bins,
    labels=cap_gain_labels,
    right=True,
)

cap_loss_bins = [-0.5, 0.5, 1000, 10000, np.inf]
cap_loss_labels = ["0", "1-1000", "1001-10000", ">10000"]
df["capital_loss_bucket"] = pd.cut(
    df["capital-loss"],
    bins=cap_loss_bins,
    labels=cap_loss_labels,
    right=True,
)

# 4.4 Occupation / workclass
exec_prof_list = ["Exec-managerial", "Prof-specialty"]
df["is_exec_prof"] = df["occupation"].isin(exec_prof_list).astype(int)

def map_workclass(w):
    if pd.isna(w):
        return "Unknown"
    w = w.strip()
    lw = w.lower()
    if "gov" in lw:
        return "Gov"
    if "self" in lw:
        return "Self-emp"
    if w == "Private":
        return "Private"
    return "Other"

df["workclass_group"] = df["workclass"].apply(map_workclass)

# 4.5 Marital / relationship features
df["is_married"] = df["marital-status"].isin(
    ["Married-civ-spouse", "Married-AF-spouse"]
).astype(int)

df["is_in_couple"] = df["relationship"].isin(["Husband", "Wife"]).astype(int)
df["has_children"] = df["relationship"].isin(["Own-child"]).astype(int)

# 4.6 Geography
df["is_US"] = (df["native-country"] == "United-States").astype(int)

europe = {
    "England","Germany","Italy","France","Ireland","Scotland","Holand-Netherlands",
    "Portugal","Greece","Yugoslavia","Hungary"
}
latin_america = {
    "Mexico","Puerto-Rico","Cuba","Jamaica","Haiti",
    "Dominican-Republic","Guatemala","Nicaragua","El-Salvador","Honduras",
    "Peru","Ecuador","Columbia","Trinadad&Tobago"
}
asia = {
    "India","Iran","Philippines","Vietnam","Laos","Cambodia","Thailand",
    "China","Japan","Hong","Taiwan","South"
}

def map_country_region(c):
    if pd.isna(c):
        return "Other"
    c = c.strip()
    if c == "United-States":
        return "US"
    if c in europe:
        return "Europe"
    if c in latin_america:
        return "LatAm"
    if c in asia:
        return "Asia"
    return "Other"

df["country_region"] = df["native-country"].apply(map_country_region)

# 4.7 Interactions
df["high_edu_mid_age"] = (
    (df["high_education_flag"] == 1) &
    df["age"].between(35, 54)
).astype(int)

df["exec_prof_long_hours"] = (
    (df["is_exec_prof"] == 1) &
    (df["long_hours_flag"] == 1)
).astype(int)

df["investor_flag"] = (
    (df["has_capital_gain"] == 1) |
    (df["has_capital_loss"] == 1)
).astype(int)

df["married_and_exec"] = (
    (df["is_married"] == 1) &
    (df["is_exec_prof"] == 1)
).astype(int)

df["married_and_high_education"] = (
    (df["is_married"] == 1) &
    (df["high_education_flag"] == 1)
).astype(int)

df["young_high_hours"] = (
    (df["age"] < 35) &
    (df["long_hours_flag"] == 1)
).astype(int)

# ============================================
# 5. Keep only labeled rows & define target
# ============================================
df_model = df[~df["income"].isna()].copy()
df_model["income_high"] = (df_model["income"] == ">50K").astype(int)

df = df_model  # modeling dataframe with labeled rows only

# ============================================
# 6. Train/valid/test split, cat_idx, class_weights
# ============================================
target = "income_high"

X = df.drop(columns=[target, "income", "Name"], errors="ignore")
y = df[target]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.40,
    stratify=y,
    random_state=42,
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42,
)

cat_cols = X.select_dtypes(include=["object", "category"]).columns
cat_idx = [X.columns.get_loc(c) for c in cat_cols]

pos_rate = y_train.mean()
neg_rate = 1.0 - pos_rate
pos_weight = neg_rate / pos_rate
class_weights = [1.0, pos_weight]

print("Train size:", X_train.shape[0])
print("Valid size:", X_valid.shape[0])
print("Test size :", X_test.shape[0])
print("Pos rate  :", pos_rate)
print("pos_weight:", pos_weight)

# ============================================
# 7. GRID SEARCH (coarse)
# ============================================
base_cat = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="PRAUC",
    iterations=600,              # a bit lower for faster grid search
    class_weights=class_weights,
    random_seed=42,
    verbose=False,
)

param_grid = {
    "depth": [4, 6, 8, 10,12],
    "learning_rate": [0.03, 0.05, 0.1],
    "l2_leaf_reg": [1, 3, 5, 7,12,16],
}

pr_auc_scorer = make_scorer(average_precision_score, needs_proba=True)

grid = GridSearchCV(
    estimator=base_cat,
    param_grid=param_grid,
    scoring=pr_auc_scorer,
    cv=3,
    n_jobs=-1,
    verbose=2,
)

grid.fit(X_train, y_train, cat_features=cat_idx)

print("Best params from grid search:")
print(grid.best_params_)
print("Best CV PR-AUC:", grid.best_score_)

best_grid_model = grid.best_estimator_

y_valid_pred_grid = best_grid_model.predict_proba(X_valid)[:, 1]
print("VALID PR-AUC (grid best):", average_precision_score(y_valid, y_valid_pred_grid))
print("VALID ROC-AUC (grid best):", roc_auc_score(y_valid, y_valid_pred_grid))

best_grid_params = grid.best_params_

# ============================================
# 8. OPTUNA TUNING (finer)
# ============================================
def objective(trial):
    base_depth = best_grid_params.get("depth", 6)
    base_lr = best_grid_params.get("learning_rate", 0.05)
    base_l2 = best_grid_params.get("l2_leaf_reg", 3.0)

    depth = trial.suggest_int("depth", max(3, base_depth - 2), base_depth + 2)
    learning_rate = trial.suggest_float("learning_rate", base_lr / 3, base_lr * 3, log=True)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", base_l2 / 10, base_l2 * 10, log=True)
    iterations = trial.suggest_int("iterations", 600, 2000)

    model = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="PRAUC",
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        class_weights=class_weights,
        random_seed=42,
        verbose=False,
        od_type="Iter",
        od_wait=100,
    )

    model.fit(
        X_train, y_train,
        eval_set=(X_valid, y_valid),
        cat_features=cat_idx,
        verbose=False,
    )

    y_valid_proba = model.predict_proba(X_valid)[:, 1]
    pr_auc = average_precision_score(y_valid, y_valid_proba)
    return pr_auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\nBest Optuna trial:")
print("  Trial number :", study.best_trial.number)
print("  Best PR-AUC  :", study.best_value)
print("  Best params  :", study.best_params)

# ============================================
# 9. FINAL MODEL WITH BEST OPTUNA PARAMS
# ============================================
best_params = study.best_params.copy()

final_model = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="PRAUC",
    iterations=best_params.get("iterations", 1000),
    depth=best_params.get("depth", 6),
    learning_rate=best_params.get("learning_rate", 0.05),
    l2_leaf_reg=best_params.get("l2_leaf_reg", 3.0),
    class_weights=class_weights,
    random_seed=42,
    verbose=100,
    od_type="Iter",
    od_wait=100,
)

X_train_full = pd.concat([X_train, X_valid], axis=0)
y_train_full = pd.concat([y_train, y_valid], axis=0)

final_model.fit(
    X_train_full, y_train_full,
    eval_set=(X_test, y_test),
    cat_features=cat_idx,
    verbose=100,
)

y_test_proba = final_model.predict_proba(X_test)[:, 1]
print("\nFINAL TEST PR-AUC:", average_precision_score(y_test, y_test_proba))
print("FINAL TEST ROC-AUC:", roc_auc_score(y_test, y_test_proba))


Train size: 23797
Valid size: 7933
Test size : 7933
Pos rate  : 0.28259864688826325
pos_weight: 2.538587360594795
Fitting 3 folds for each of 90 candidates, totalling 270 fits
[CV] END .........depth=4, l2_leaf_reg=1, learning_rate=0.05; total time= 1.1min
[CV] END ..........depth=4, l2_leaf_reg=1, learning_rate=0.1; total time= 1.1min
[CV] END .........depth=4, l2_leaf_reg=3, learning_rate=0.03; total time= 1.0min
[CV] END .........depth=4, l2_leaf_reg=3, learning_rate=0.05; total time= 1.1min
[CV] END .........depth=4, l2_leaf_reg=5, learning_rate=0.03; total time= 1.0min
[CV] END .........depth=4, l2_leaf_reg=5, learning_rate=0.05; total time= 1.0min
[CV] END ..........depth=4, l2_leaf_reg=5, learning_rate=0.1; total time= 1.0min
[CV] END .........depth=4, l2_leaf_reg=7, learning_rate=0.03; total time= 1.0min
[CV] END ..........depth=4, l2_leaf_reg=7, learning_rate=0.1; total time= 1.1min
[CV] END ........depth=4, l2_leaf_reg=12, learning_rate=0.03; total time= 1.0min
[CV] END .....

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END .........depth=4, l2_leaf_reg=1, learning_rate=0.03; total time= 1.0min
[CV] END .........depth=4, l2_leaf_reg=1, learning_rate=0.05; total time= 1.1min
[CV] END ..........depth=4, l2_leaf_reg=1, learning_rate=0.1; total time= 1.1min
[CV] END .........depth=4, l2_leaf_reg=3, learning_rate=0.05; total time= 1.0min
[CV] END ..........depth=4, l2_leaf_reg=3, learning_rate=0.1; total time= 1.0min
[CV] END .........depth=4, l2_leaf_reg=5, learning_rate=0.03; total time= 1.0min
[CV] END ..........depth=4, l2_leaf_reg=5, learning_rate=0.1; total time= 1.1min
[CV] END .........depth=4, l2_leaf_reg=7, learning_rate=0.03; total time= 1.0min
[CV] END .........depth=4, l2_leaf_reg=7, learning_rate=0.05; total time= 1.1min
[CV] END ........depth=4, l2_leaf_reg=12, learning_rate=0.03; total time= 1.0min
[CV] END ........depth=4, l2_leaf_reg=12, learning_rate=0.05; total time= 1.1min
[CV] END .........depth=4, l2_leaf_reg=12, learning_rate=0.1; total time= 1.1min
[CV] END ........depth=4, l2

[I 2025-11-30 13:38:24,367] A new study created in memory with name: no-name-c5ae6c82-64bc-4fc5-8baa-c271552b1da6


Best params from grid search:
{'depth': 4, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
Best CV PR-AUC: 0.8443029234610009
VALID PR-AUC (grid best): 0.8526049923869239
VALID ROC-AUC (grid best): 0.9287289225001336


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-11-30 13:38:49,943] Trial 0 finished with value: 0.8539276867002019 and parameters: {'depth': 6, 'learning_rate': 0.08993055075400123, 'l2_leaf_reg': 0.8638485642765822, 'iterations': 1993}. Best is trial 0 with value: 0.8539276867002019.
[CV] END .........depth=8, l2_leaf_reg=5, learning_rate=0.03; total time= 2.3min
[CV] END .........depth=8, l2_leaf_reg=5, learning_rate=0.05; total time= 2.4min
[CV] END .........depth=8, l2_leaf_reg=7, learning_rate=0.03; total time= 2.3min
[CV] END .........depth=8, l2_leaf_reg=7, learning_rate=0.05; total time= 2.3min
[CV] END ..........depth=8, l2_leaf_reg=7, learning_rate=0.1; total time= 2.4min
[CV] END ........depth=8, l2_leaf_reg=12, learning_rate=0.05; total time= 2.3min
[CV] END .........depth=8, l2_leaf_reg=12, learning_rate=0.1; total time= 2.4min
[CV] END ........depth=8, l2_leaf_reg=16, learning_rate=0.03; total time= 2.3min
[CV] END .........depth=8, l2_leaf_reg=16, learning_rate=0.1; total time= 2.4min
[CV] END ........depth=1